In [ ]:
# load the data - download in matlab format from:
# https://www.nist.gov/itl/iad/image-group/emnist-dataset
import scipy.io
mat = scipy.io.loadmat('../Dataset/emnist-letters.mat')
d = mat['dataset']
images = d[0,0]['train'][0,0]['images']
labels = d[0,0]['train'][0,0]['labels']
test_images = d[0,0]['test'][0,0]['images']
test_labels = d[0,0]['test'][0,0]['labels']

In [ ]:
import numpy as np

n= images.shape[0]
# format training/testing sets
images_2 = np.asarray(images[0:n][:],dtype=np.float64)/255
labels_2 = np.asarray(labels[0:n],dtype=np.int32).ravel();
test_images_2 = np.asarray(test_images,dtype=np.float64)/255
test_labels_2 = np.asarray(test_labels,dtype=np.int32).ravel();

# perform dimensionality reduction
from sklearn.decomposition import PCA
ncomps = 25; # We'll reduce from 784 dimensions to 25
pca = PCA(n_components=ncomps);
pca.fit(images_2)
# Those 25 dimensions still explain a lot of the variance
print 'Explained Variance: ' + str(round(100*sum(pca.explained_variance_ratio_))) + '%'
mat = pca.components_ #This is the matrix that can be used to perform the transformation in the app

images_pca = pca.transform(images_2)
test_images_pca = pca.transform(test_images_2)

In [ ]:
# learn the model
import time
start = time.time()

nsamps = 20000
from sklearn import svm
clf = svm.SVC(probability=True)
clf.fit(images_pca[:nsamps][:],labels_2[:nsamps])

end = time.time()
print( "Time it took to learn the model: " + str(end - start) + " seconds")

In [ ]:
start = time.time()
answers = clf.predict(test_images_pca)
end = time.time()
print( "Time it took to test the model: " + str(end - start) + " seconds")

In [ ]:
cnt = 0;
s=[i==j for i,j in zip(test_labels_2,answers)]
incorrect=[i!=j for i,j in zip(test_labels_2,answers)]
print("Percent Correct: " + str(100*sum(s)/len(s)) + "%")

In [ ]:
import coremltools
coreml_model = coremltools.converters.sklearn.convert(clf, "convertedPCAValues", output_feature_names='letterIndex')
coreml_model.author = 'Kate Bonnen and Conrad Stoll'
coreml_model.license = 'MIT'
coreml_model.short_description = "Recognize the hand-drawn letter from an input image."

coreml_model.input_description['convertedPCAValues'] = 'The input image alpha values multiplied by a PCA matrix.'

coreml_model.output_description['letterIndex'] = 'Most likely letter index, ranging from 1 to 26.'
coreml_model.output_description['classProbability'] = 'The probability of each letter index.'

coreml_model.save('../Models/letters_pca.mlmodel')